In [1]:
import pandas as pd

# 1. 特征：第一列是节点 ID，其余是特征
feature = pd.read_csv("./data/BUPT/TF.features",
                      header=None,
                      sep=r'\s+',            # 兼容多空格/制表符
                      engine='python',
                      dtype=str)
feature = feature.rename(columns={0: 'node_id'})
# 其余列统一命名成 f0,f1,...
feat_cols = {i: f'f{i-1}' for i in range(1, feature.shape[1])}
feature.rename(columns=feat_cols, inplace=True)

# 2. 标签：第一列节点 ID，第二列标签
labels = pd.read_csv("./data/BUPT/TF.labels",
                     header=None,
                     sep=r'\s+',
                     engine='python',
                     dtype=str,
                     names=['node_id', 'label'])

# 3. 边：第一列 src，第二列 target
edges = pd.read_csv("./data/BUPT/TF.edgelist",
                    header=None,
                    sep=r'\s+',
                    engine='python',
                    dtype=str,
                    names=['src', 'target'])

In [2]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116383 entries, 0 to 116382
Data columns (total 40 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   node_id  116383 non-null  object
 1   f0       116383 non-null  object
 2   f1       116383 non-null  object
 3   f2       116383 non-null  object
 4   f3       116383 non-null  object
 5   f4       116383 non-null  object
 6   f5       116383 non-null  object
 7   f6       116383 non-null  object
 8   f7       116383 non-null  object
 9   f8       116383 non-null  object
 10  f9       116383 non-null  object
 11  f10      116383 non-null  object
 12  f11      116383 non-null  object
 13  f12      116383 non-null  object
 14  f13      116383 non-null  object
 15  f14      116383 non-null  object
 16  f15      116383 non-null  object
 17  f16      116383 non-null  object
 18  f17      116383 non-null  object
 19  f18      116383 non-null  object
 20  f19      116383 non-null  object
 21  f20      1

In [3]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125713 entries, 0 to 125712
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   node_id  125713 non-null  object
 1   label    125713 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [4]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226108 entries, 0 to 226107
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   src     226108 non-null  object
 1   target  226108 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [5]:
labels['label'].value_counts()

label
0    99861
3     9330
1     8448
2     8074
Name: count, dtype: int64

In [6]:
feature['node_id'].nunique()

116383

In [7]:
edges['src'].nunique()

60368

In [8]:
edges['target'].nunique()

53465

In [9]:
60368+53465

113833

In [10]:
# 所有在边上出现过的节点（src 或 target）
edge_nodes = set(edges['src']).union(set(edges['target']))

# feature 里拥有的节点
feat_nodes = set(feature['node_id'])

# 既在 feature 里又出现在边上的
common_nodes = feat_nodes & edge_nodes

print('feature 中 node_id 总数：', feature['node_id'].nunique())
print('出现在 edges 的 src 或 target 的 node_id 数：', len(common_nodes))
print(f'占比：{len(common_nodes) / feature["node_id"].nunique():.2%}')

feature 中 node_id 总数： 116383
出现在 edges 的 src 或 target 的 node_id 数： 70111
占比：60.24%


In [11]:
feat_cols = [c for c in feature.columns if c != 'node_id']
feature[feat_cols] = feature[feat_cols].apply(pd.to_numeric, errors='coerce')

# 全 0 行
all_zero_rows = (feature[feat_cols] == 0).all(axis=1)

print('全 0 特征的行数：', all_zero_rows.sum())
print('占比：', f'{all_zero_rows.mean():.2%}')

全 0 特征的行数： 0
占比： 0.00%


In [12]:
# 确保 ID 类型一致（字符串）
node_of_interest = '99992'
edges['src'] = edges['src'].astype(str)
edges['target'] = edges['target'].astype(str)

# 总出现次数
total = (edges['src'] == node_of_interest).sum() + (edges['target'] == node_of_interest).sum()

# 分别统计
src_cnt  = (edges['src'] == node_of_interest).sum()
tgt_cnt  = (edges['target'] == node_of_interest).sum()

print(f'节点 {node_of_interest} 在 edges 中共出现 {total} 次')
print(f'  作为 src 出现 {src_cnt} 次')
print(f'  作为 target 出现 {tgt_cnt} 次')

节点 99992 在 edges 中共出现 0 次
  作为 src 出现 0 次
  作为 target 出现 0 次


In [13]:
# 构造双向元组集合
edge_set  = set(map(tuple, edges[['src','target']].values))
rev_set   = set(map(tuple, edges[['target','src']].values))

# 如果对称，则 edge_set 与 rev_set 应该几乎完全重合
sym_rate = len(edge_set & rev_set) / len(edge_set)
print(f'对称率={sym_rate:.2%}')
if sym_rate > 0.9:
    print('→ 基本对称（可视为无向图）')
else:
    print('→ 明显不对称（有向图）')

对称率=47.08%
→ 明显不对称（有向图）


In [15]:
import numpy as np

In [16]:
# 把 src-target 整行排序后去重；如果去重后行数减半 → 对称
edges[['src','target']] = np.sort(edges[['src','target']].values, axis=1)
dup = edges.duplicated()          # 重复行
print('对称（无向）' if dup.mean() > 0.45 else '不对称（有向）')

不对称（有向）


In [17]:
# 确保字符串类型
edges['src'] = edges['src'].astype(str)
edges['target'] = edges['target'].astype(str)

# 把所有有向边放进一个集合
edge_set = set(map(tuple, edges[['src', 'target']].values))

# 统计同时存在反向边的数目
bi_edges = [(u, v) for u, v in edge_set if (v, u) in edge_set]

# 去重：无向意义下只算一条
bi_count = len(bi_edges) // 2

print('双向边（无向意义）数量：', bi_count)
print('占全部边比例：', f'{bi_count / len(edges):.2%}')

双向边（无向意义）数量： 421
占全部边比例： 0.19%


In [18]:
# 先确保两表 ID 类型一致
feature['node_id'] = feature['node_id'].astype(str)
labels['node_id']  = labels['node_id'].astype(str)

# 检测
missing = set(feature['node_id']) - set(labels['node_id'])

print('feature 中是否全部 node_id 都在 labels 里：', len(missing) == 0)
if missing:
    print('缺失的 node_id 数量：', len(missing))
    print('前 10 个例子：', list(missing)[:10])
else:
    print('✅ 全部匹配')

feature 中是否全部 node_id 都在 labels 里： True
✅ 全部匹配


In [ ]:
feature['node_id'].